In [1]:
import numpy as np
import pandas as pd

# set screen variables for simulation

In [2]:
#Number of tested compounds (default: 10T)

tested = 100000

#Assumed Hitrate TPO (default: 8%)

Hitrate_TPO = 0.08

# This is the hitrate of aromatase!!!
#Hitrate_TPO = 0.10

#Assumed Hitrate desired biological activity (default: 1%)

Hitrate_Bio = 0.01
# Assumed Project hitrate
#Hitrate_Bio = 0.7

# set model variables for simulation

In [5]:
data = {
    'Model': ['gbt', 
              'catboost'], 
              #'mother', 
              #'Aromatase'], 
    'recall': [0.4, 
               0.8], 
               #0.31, 
               #0.39], 
    'precision':[0.8, 
                 0.40] 
                # 0.85, 
                # 0.80]
}

df = pd.DataFrame.from_dict(data)

precision =0.8560606060606061
recall =0.31301939058171746
TP =0.05006645990252548
FP =0.008418254319893664
TN =0.8316349136021267
FN =0.10988037217545414

# calculate relevant metrics of screen without model

In [6]:
## Intersect TPO/Biol (assumption: variables are independent of each other this can be skipped)

## All TPO hits
all_TPO = tested*Hitrate_TPO
print('all TPO-hits:'+str(all_TPO))

## All compounds with desired biological activity
all_bio = tested*Hitrate_Bio
print('all cpds with desired biological activity:'+ str(all_bio))

#Intersect_TPO_BIO: Bio and TPO active --> biologically active compounds we throw away

Intersect_TPO_Bio = (Hitrate_Bio * Hitrate_TPO)*tested
print('Intersect_TPO_Bio:'+ str(Intersect_TPO_Bio))

#desired hits: TPO inactive, bioactive

desired_hits = (Hitrate_Bio * (1-Hitrate_TPO))*tested
print('desired_hits:'+ str(desired_hits))

# not biologically active TPO-hits

TPO_active_bio_not = ((1-Hitrate_Bio) * (Hitrate_TPO))*tested
print('not biologically active TPO-hits:'+ str(TPO_active_bio_not))

# TPO und Bio nicht aktiv

TPO_Bio_inactive = ((1-Hitrate_Bio) * (1-Hitrate_TPO))*tested
print('inactive biologically & TPO:'+ str(TPO_Bio_inactive))

# Sanity test
all = Intersect_TPO_Bio + desired_hits + TPO_active_bio_not + TPO_Bio_inactive
print (all==tested)

all TPO-hits:8000.0
all cpds with desired biological activity:1000.0
Intersect_TPO_Bio:80.0
desired_hits:920.0
not biologically active TPO-hits:7920.000000000001
inactive biologically & TPO:91080.0
True


# notebook utils

In [7]:
def calc_TP(recall:float, hitrate:float, tested:float) -> float:
    ''' How many true positives does a model with a given recall produce for a fixed number of tested compounds: 
    FP + TP = Hitrate_TPO
    recall = TP / Hitrate TPO --> TP = recall * Hitrate TPO'''    
    TP = recall*hitrate*tested    
    return TP

def calc_FP(precision:float,TP:float, tested:float) -> float:
    '''How many false positives does a model with a given precision produce for a fixed number of tested compounds:
    precision = TP/(TP + FP) --> FP = (TP/precision)-TP    '''
    FP = (TP/(precision))-TP    
    return FP

def calc_FN(recall:float, TP:float)-> float:
    '''How many false negatives are produced by a model with a given recall:
    recall = TP/(TP+FN) --> FN = (TP/recall)-TP'''    
    FN = (TP/recall)-TP    
    return FN

def calc_TN(TP:float, FP:float, FN:float, tested:float)-> float:
    '''How many true negatives are produced by a model with a given precision and recall (i.e. TP, FP, FN and tested known)'''    
    TN = tested - (TP+FP+FN)    
    return TN

In [8]:
# If models are applied only substances that are predicted safe are allowed for biological screening (i.e. FN, TN). TN and TP are removed and replaced by more compounds that are predicted safe from the "infinite" screening pool.

def TPO_hit_assigned(TP:float, FP:float)-> float:
    pred_hit = TP + FP
    return pred_hit

def TPO_safe_assigned(pred_hit:float, tested:float)-> float:
    pred_safe = tested - pred_hit
    return pred_safe

def calc_desired_hits_model(TN:float, Hitrate_Bio:float)-> float:
    '''These are desired hits allowed for testing if model is applied: no TPO inhibitor but biologically active'''
    desired_hits_model = (TN*Hitrate_Bio)
    return desired_hits_model

def calc_TPO_hits_model(FN:float, Hitrate_Bio:float)-> float:
    '''These are TPO hits with desired biological activbity allowed for screening even though model was applied'''
    TPO_hits_model = (FN*Hitrate_Bio)
    return TPO_hits_model

def calc_desired_hits_missed(FP:float, Hitrate_Bio:float)-> float:
    '''These are desired hits that are filtered out because model was applied'''
    desired_hits_missed = (FP*Hitrate_Bio)
    return desired_hits_missed

def calc_desired_hit_number_after_model_use(desired_hits_model:float, pred_safe:float, tested:float)-> float:
    '''If model is used we can expect a hitrate given by number of desired hits divided by number of allowed compounds; This is expeced to stay constant if screen is filled up with more compounds predicted safe'''
    hit_number = (desired_hits_model/pred_safe)*tested
    return hit_number

def calc_TPO_hits_not_removed(TPO_hits_model:float, pred_safe:float, tested:float)-> float: 
    bad_hit_number = (TPO_hits_model/pred_safe)*tested
    return bad_hit_number

In [9]:
# sanity checks: calculate precision from derived values, calculate number of tested compounds from derived values
def calc_precision (TP:float, FP:float, tested:float)-> float:
    precision = ((TP/(TP + FP)))
    return precision

def calc_tested(pred_hit:float, pred_safe:float)-> float:
    calc_tested = pred_hit + pred_safe
    return calc_tested

# Calculation

In [10]:
# calculate confusion matrix for a given dataset and given model recall/precision
df['TP'] = df.apply(lambda row: calc_TP(row['recall'], hitrate = Hitrate_TPO, tested = tested), axis=1)
df['FP'] = df.apply(lambda row: calc_FP(row['precision'], row['TP'], tested = tested), axis=1)
df['FN'] = df.apply(lambda row: calc_FN(row['recall'], row['TP']), axis=1)
df['TN'] = df.apply(lambda row: calc_TN(row['TP'], row['FP'],  row['FN'], tested = tested), axis=1)

In [11]:
# calculate effect of model use on screen
df['pred_hit'] = df.apply(lambda row: TPO_hit_assigned(row['FP'], row['TP']), axis=1)
df['pred_safe'] = df.apply(lambda row: TPO_safe_assigned(row['pred_hit'], tested = tested), axis=1)
df['desired_hits_model'] = df.apply(lambda row: calc_desired_hits_model(row['TN'], Hitrate_Bio), axis=1)
df['TPO_hits_model'] = df.apply(lambda row: calc_TPO_hits_model(row['FN'], Hitrate_Bio), axis=1)
df['desired_hits_missed'] = df.apply(lambda row: calc_desired_hits_missed(row['FP'], Hitrate_Bio), axis=1)
df['hit_number'] = df.apply(lambda row: calc_desired_hit_number_after_model_use(row['desired_hits_model'], row['pred_safe'], tested), axis=1)
df['bad_hit_number'] = df.apply(lambda row: calc_TPO_hits_not_removed(row['TPO_hits_model'], row['pred_safe'], tested), axis=1)

In [12]:
# calculate sanity checks
df['calc_precision'] = df.apply(lambda row: calc_precision(row['TP'], row['FP'], tested = tested), axis=1)
df['calc_tested'] = df.apply(lambda row: calc_tested(row['pred_hit'], row['pred_safe']), axis=1)
df['sum_TP_FP_TN_FN']= df.apply(lambda row:(row['TP'] + row['FP'] + row['TN'] + row['FN']), axis=1)

In [13]:
df

,Model,recall,precision,TP,FP,FN,TN,pred_hit,pred_safe,desired_hits_model,TPO_hits_model,desired_hits_missed,hit_number,bad_hit_number,calc_precision,calc_tested,sum_TP_FP_TN_FN
0,gbt,0.4,0.8,3200.0,800.0,4800.0,91200.0,4000.0,96000.0,912.0,48.0,8.0,950.000000,50.000000,0.8,100000.0,100000.0
1,catboost,0.8,0.4,6400.0,9600.0,1600.0,82400.0,16000.0,84000.0,824.0,16.0,96.0,980.952381,19.047619,0.4,100000.0,100000.0


In [42]:
#precision =0.8560606060606061
#recall =0.31301939058171746
TP_check =(0.05006645990252548*4514)
FP_check =(0.008418254319893664*4514)
TN_check =(0.8316349136021267*4514)
FN_check =(0.10988037217545414*4514)

In [43]:
print(TP_check)
print(FP_check)
print(FN_check)
print(TN_check)

226.0
38.0
496.0
3754.0
